<a href="https://colab.research.google.com/github/linccolnevans/datascience/blob/main/LincEvanUnit5Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

You need to submit a .ipynb file with the first four letters of one first name, the first four letters of one last name, and U5proj.

Ex.

A student named Avy Aguacate Avocado Palta will submit a file named "Avy_AvocU5proj.ipynb", since she normally just goes by Avy, and, when pressed, reports Avocado as her last name. The " _ " after "Avy" serves as the fourth "letter" in her first name.

Ex2.

Army Armadillo will submit a file named "ArmyArmaU5proj.ipynb"

Example Dataset: Titanic Survival Data (One Predictor)

Description: The dataset contains information about Titanic passengers, such as age, class, sex, and survival status. In this case, we will focus on just one predictor.
Response Variable: Survived (binary: 0 = No, 1 = Yes).
Predictor: Sex (categorical: Male or Female).
GLM Type: Logistic Regression (using a binomial distribution).
Usage: To predict the probability of survival based solely on the gender of the passengers.

# Unit 5 Project: Making a Generalized Linear Model (1 predictor)

**Task**:

Two parts:

1. Propose a data set that's compatible with a generalized linear model structure built with pymc. I must approve your submission before you move on to part 2.

2. Create, analyze, and present a generalized linear model using pymc within the colab environment, on a data set of your choice.



Guidlines:

- You are allowed to copy and paste any code from the notes, including my answer key (AK), without any reference or citations. Same goes for the course textbook, BAP3.

- This is an individual project. It is fine if you model the same process as a peer, but your raw data **must** be substantially different. "Substantially different" means that I, your instructor, arbitrarily think your data sets are different enough.

- Do not look at other student's code/jupyter notebooks. Copying, or copying and pasting, other students code will result in a failing grade. Also, I will tell all the other teachers I know that you cheated.

- If you want to know what another student did, ask them--any verbal communication about a project is allowed and encouraged. If you want to see a graphic they made, ask them to sketch a picture of it for you by hand.

- Any handwritten communication is also allowed and encouraged, so long as all parties are in the same physical location together. Example: you may sketch out a piece of code while verbally explaining to someone what you did. Do NOT: hand write the code for your entire project, take a picture, and send it to everyone in the course.

- If you use someone's code from outside this course (like from the internet, an AI, or your parent), you must ask me if it is ok, and you must cite them in your work. I don't care all they did was to click your space bar one more time: you tell me, and you cite their contribution. You don't need to tell me about or cite verbal or handwritten communication though.

- You may use a more complicated model--like something from further along in the course--IF I give you permission.

- If you are unsure whether or not something is ok or allowed: ASK BEFORE YOU DO IT.

Faliure to adhere to these guidlines will result in a failing grade.

Tips:

- Try to pick something that you care about, have knowledge of, experience with, or that you find interesting/cool.

- Gather data *before* picking a likelihood.

-  Use your best idea now; maybe you can look at the context you used in the previous project, and gather different data from it.

- Linear models are generalized linear models. This is not a competition to come up with the most complicated heirearchical glm. **Find a data set that interests you, and apply the simplest model that captures the patterns in the data.**

- Comment heavily, and explain your steps and thought process with text blocks. This is not only requried, it'll also help you understand what you're doing and what you want to do.

- Want an example? The Unit5NotesAK *is* your example! You are expected to be somewhere between a half and three-quarters as detailed--but more is always welcome.

- Its ok to pick a data set for which you don't qute understand the context--so long as you think the context is cool.


my data set is batting sweet spot percent compared to hard hit percent, to see if the players that hit the ball with the best prat of the bat on average swing the hardest and put the ball into play at a high speed.

In [1]:

import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd
import arviz as az
import xarray as xr
import pymc as pm

from scipy.interpolate import PchipInterpolator

In [2]:
url_baseball = 'https://github.com/linccolnevans/datascience/blob/78818e24a430c887cc516b8f1c373212c95b7800/baseballstats.csv'

In [9]:
baseball = pd.read_csv(url_baseball)

ParserError: Error tokenizing data. C error: Expected 1 fields in line 42, saw 30


In [6]:
baseball

NameError: name 'baseball' is not defined

In [3]:
baseball.plot(x="sweet_spot_percent", y="hard_hit_percent", figsize=(12, 3), kind="scatter")

NameError: name 'baseball' is not defined

In [8]:
with pm.Model() as model_lb:

    #priors for the linear part of our model
    α = pm.Normal("α", mu=0, sigma=100)
    β = pm.Normal("β", mu=0, sigma=10)

    #this is the linear part of our model
    μ = pm.Deterministic("μ", α + β * baseball.sweet_spot_percent)

    #prior for the standard deviation of our likelihood
    #Cauchy is a T dist with nu = 1
    σ = pm.HalfCauchy("σ", 10)

    #likelihood
    y_pred = pm.Normal("y_pred", mu=μ, sigma=σ, observed=baseball.hard hit percent)

    #inference data object
    idata_lb = pm.sample()

SyntaxError: invalid syntax. Perhaps you forgot a comma? (<ipython-input-8-1805ef8968c9>, line 15)

In [7]:
pm.model_to_graphviz(model_lb)

NameError: name 'model_lb' is not defined

In [ ]:
az.plot_posterior(idata_lb, var_names=["~μ"], figsize=(12, 3))

In [ ]:
posterior = az.extract(idata_lb, num_samples=100)

# grabbing x values for graphing.
x_plot = xr.DataArray(
    np.linspace(baseball.sweet spot percent.min(), baseball.sweet spot percent.max(), 50),
    dims="plot_id"
    )

# this creates the expected line, the path we predict temperature and
#rented bike count deviates from
mean_line = posterior["α"].mean() + posterior["β"].mean() * x_plot

#These are a 50 lines that our model came up with
lines = posterior["α"] + posterior["β"] * x_plot

#set up plot
_, ax = plt.subplots()

#plots 50 other lines our model came up with.
ax.plot(x_plot, lines.T, c="C1", alpha=0.2, label="lines")

#plots the mean line
ax.plot(x_plot, mean_line, c="C0", label="mean line")

#plot the raw data
ax.plot(baseball.sweet spot percent, baseball.hard hit percent, "C2.", zorder=-3)

#label axes and create legend
ax.set_xlabel("sweet spot percent")
ax.set_ylabel("hard hit percent")

In [ ]:
pm.sample_posterior_predictive(idata_lb, model=model_lb,  extend_inferencedata=True)

In [ ]:
#plots the posterior predictive distribution

#grabs the expected line, or line of best fit formula
mean_line = idata_lb.posterior["μ"].mean(("chain", "draw"))

#creates some x values to run through our line formula
sweet spot percent = np.random.normal(baseball.sweet spot percent.values, 0.01)
idx = np.argsort(sweet spot percent)
x = np.linspace(sweet spot percent.min(), sweet spot percent.max(), 15)

#grabs the 94% HDI and 50% HDI, and sets them up for graphing.
y_pred_q = idata_lb.posterior_predictive["y_pred"].quantile(
    [0.03, 0.97, 0.25, 0.75], dim=["chain", "draw"]
)
y_hat_bounds = iter(
    [
        PchipInterpolator(sweet spot percent[idx], y_pred_q[i][idx])(x)
        for i in range(4)
    ]
)

#plots raw data and our line of best fit
_, ax = plt.subplots()
ax.plot(baseball.sweet spot percent, baseball.hard hit percent, "C2.", zorder=-3)
ax.plot(baseball.sweet spot percent[idx], mean_line[idx], c="C0")


#graphs the 94% and 50% HDIs
for lb, ub in zip(y_hat_bounds, y_hat_bounds):
    ax.fill_between(x, lb, ub, color="C1", alpha=0.5)

#labels
ax.set_xlabel("sweet spot percent")
ax.set_ylabel("hard hit percent")

In [ ]:
az.plot_ppc(idata_lb, num_pp_samples=200, colors=["C1", "C0", "C1"])

In [ ]:
sns.violinplot(baseball.hard hit percent)

In [ ]:
with pm.Model() as model_nbb:

    #priors for the linear part of the model
    α = pm.Normal("α", mu=0, sigma=1)
    β = pm.Normal("β", mu=0, sigma=10)

    #the linear part of our model,
    #but with a twist:
    #our line is exponentiated, in order to make our all our values positive
    μ = pm.Deterministic("μ", pm.math.exp(α + β * baseball.sweet spot percent))

    #prior for the likelihood's standard deviation
    σ = pm.HalfNormal("σ", 10)

    #likelihood
    y_pred = pm.NegativeBinomial("y_pred", mu=μ, alpha=σ, observed=baseball.hard hit percent)

    idata_nbb = pm.sample()

In [ ]:
pm.sample_posterior_predictive(idata_nbb, model = model_nbb, extend_inferencedata=True)

In [ ]:
az.plot_ppc(idata_nbb, num_pp_samples=200, colors=["C1", "C0", "C1"])

In [ ]:
mean_line = idata_nbb.posterior["μ"].mean(("chain", "draw"))
sweet spot percent = np.random.normal(baseball.sweet spot percent.values, 0.01)
idx = np.argsort(sweet spot percent)
x = np.linspace(sweet spot percent.min(), sweet spot percent.max(), 15)
y_pred_q = idata_nbb.posterior_predictive["y_pred"].quantile(
    [0.03, 0.97, 0.25, 0.75], dim=["chain", "draw"]
)
y_hat_bounds = iter(
    [
        PchipInterpolator(sweet_spot_percent[idx], y_pred_q[i][idx])(x)
        for i in range(4)
    ]
)

_, ax = plt.subplots()
ax.plot(baseball.sweet spot percent, baseball.hard hit percent, "C2.", zorder=-3)
ax.plot(baseball.sweet spot percent[idx], mean_line[idx], c="C0")

for lb, ub in zip(y_hat_bounds, y_hat_bounds):
    ax.fill_between(x, lb, ub, color="C1", alpha=0.5)

ax.set_xlabel("sweet spot percent")
ax.set_ylabel("hard hit percent")

Through this project I learned that a lot of the hardest hitters dont need to have a high sweet spot rate in order to hit bombs! Oneil Cruz has a 22.8 percent differential between his sweet spot and hard hit percentages, and I dont think thats the biggest difference in the league. There are a lot of players that swing hard every time and can hit homeruns, but I always thought those homeruns came from sweet spots only, however some of this data disproves that.